<a href="https://colab.research.google.com/github/daleas0120/Metallurgy/blob/main/Heats_of_Formation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heats of Formation for Ternary Alloys

Implementation based on *A Thermodynamic Approach to Predict Formation Enthalpies of Ternary Systems Based on Miedema's Model* by Sadat et. al. [Citation](https://link.springer.com/article/10.1007/s11661-016-3533-4)

Excerpt from Table 1. The Chemical Potential, Electron Density, and Molar Volume Parameters of Some Elements for Calculations of Chemical Enthalpy Mixing

| $Element$ | $ϕ^*$ |$n_{ws}^{1/3}$| $V_m^{2/3}$|
| --- | ---|  --- | --- |
| Fe | 4.93 | 1.77 | 3.7 |
| Ni | 5.20 | 1.75 | 3.5 |
| V | 4.25 | 1.64 | 4.1 |



## Theoretical Equations

$$\Delta H_{ABC}^{Chem} = f(C_{ABC}^{S}) (\\x_A^{AB} \Delta H_{AinB}^{sol} + x_B^{AB} \Delta H_{BinA}^{sol} + \\ x_A^{AC} \Delta H_{AinC}^{sol} + x_C^{AC} \Delta H_{CinA}^{sol} + \\ x_B^{BC} \Delta H_{BinC}^{sol} + x_C^{BC} \Delta H_{CinB}^{sol} )$$


where 

$$ x_i^{ij} = \frac{x_i}{x_i + x_j}$$


$$ C_{A(ABC)}^S = \frac{x_A V_A^{2/3}}{x_A V_A^{2/3} + x_B V_B^{2/3} + x_C V_C^{2/3}}$$



$$ f\left ( C_{ABC}^S \right ) = \frac{1}{3} \times \left [ C_A^S C_B^S (1 + \delta(C_A^S C_B^S)^2) + C_A^S C_C^S (1 + \delta(C_A^S C_C^S)^2) + C_B^S C_C^S (1 + \delta(C_B^S C_C^S)^2) \right ]$$

$$ \Delta H_{i in j}^{sol} = \frac{2 P V_i^{2/3}}{(n_{ws}^i)^{-1/3} + (n_{ws}^j)^{-1/3}} \times \left [-(\Delta \phi^*)^2 + \frac{Q}{P} \left ( (n_{ws}^i)^{1/3} - (n_{ws}^j)^{1/3} \right ) ^2 - \frac{R}{P}\right ]$$

Furthermore: 

$$ \delta = [0, 5, 8]$$

for single-atomic (solid), amorphous alloy, or intermetallic compounds respectively and $R = 0$ unless alloying a transition with a non-transition metal. 

In [1]:
# Import Statements
import numpy as np
import pandas as pd

In [2]:
def get_deltaH_AinB(V_A, n_A, en_A, n_B, en_B, P, R):
  tmpA = 2*P*np.power(V_A, 1)/((1/n_A) + (1/n_B))
  
  deltEn = -1*(np.power(np.abs(en_A - en_B), 2))

  tmp = np.abs(np.power(n_A, 1) - np.power(n_B, 1))
  deltn = 9.4*np.power(tmp, 2)
  
  result = tmpA*(deltEn + deltn - R)

  return result


def get_E_lattice_stability(Z, struct=None):
  if struct == 'hcp':
    row = 0
  elif struct == 'fcc':
    row = 1
  elif struct == 'bcc':
    row = 2
  else:
    print('Warning: Structure does not exist in database. Returning results for FCC.')
    row = 0
  col = int(np.floor(Z-3))

  table4 = np.array([[-2.4, -2.5, 10.0, 15.0, 13.0, -5.0, -10.5, -11.0, -8.0, -1.0], 
            [-2.0, -1.5, 9.0, 14.0, 11.0, -3.0, -9.5, -11.0, -9.0, -2.0], 
            [2.2, 2.0, -9.5, -14.5, -12.0, 4.0, 100, 11.0, 8.5, 1.5]])
  
  return table4[row, col]

def get_avg_Z(atomA, xa, atomB, xb):
  Za = atomA['N']
  Zb = atomB['N']
  return xa*Za + xb*Zb

def get_H_struct(atomA, xa, atomB, xb):
  Z_alloy = get_avg_Z(atomA, xa, atomB, xb)
  E_alloy = get_E_lattice_stability(Z_alloy, 'fcc')

  Ea = get_E_lattice_stability(atomA['N'], atomA['structure'])
  Eb = get_E_lattice_stability(atomB['N'], atomB['structure'])
  
  result = E_alloy - xa*Ea - xb*Eb
  print('H_struct: ', result)
  return result


def get_deltE(Ka, Ga, Va, Kb, Gb, Vb):

  result = (2*Ka*Gb*(np.power(Va - Vb, 2)))/(3*Ka*Vb + 4*Gb*Va)
  return result

def get_deltaH_elast(atomA, xa, atomB, xb):
  
  Ka = atomA['K']
  Kb = atomB['K']
  Ga = atomA['G']
  Gb = atomB['G']
  Va = atomA['V']
  Vb = atomB['V']

  Eab = get_deltE(Ka, Ga, Va, Kb, Gb, Vb)

  Eba = get_deltE(Kb, Gb, Vb, Ka, Ga, Va)

  result = xa*xb*(xb*Eab + xa*Eba)

  return result


def get_deltaH(f, xAB_A, H_AB, xAB_B, H_BA, xAC_A, H_AC, 
               xAC_C, H_CA, xBC_B, H_BC, xBC_C, H_CB):
  result = f*(xAB_A*H_AB + xAB_B*H_BA + xAC_A*H_AC + xAC_C*H_CA + 
                xBC_B*H_BC + xBC_C*H_CB)

  return result


def get_x_AinB(x_a, x_b):
  ''' Unitless '''
  result=x_a/(x_a + x_b)
  return result


def get_f(C_A, C_B, C_C, delta):
  ''' Unitless '''
  result=(C_A*C_B*(1+delta*((C_A*C_B)**2)) + 
          C_A*C_C*(1+delta*((C_A*C_C)**2)) + 
          C_B*C_C*(1+delta*((C_B*C_C)**2)))/3.

  return result


def get_CS(x_A, V_A, x_B, V_B, x_C, V_C):
  '''
  Unitless 
  '''
  result = (x_A*np.power(V_A, 1))/(x_A*np.power(V_A, 1) + x_B*np.power(V_B, 1) + x_C*np.power(V_C, 1))

  return result

def get_H(atom_a, xa, atom_b,xb, atom_c, xc, P, R, delta):
  V_a = atom_a['V']
  V_b = atom_b['V']
  V_c = atom_c['V']

  n_a = atom_a['n']
  n_b = atom_b['n']
  n_c = atom_c['n']

  en_a = atom_a['en']
  en_b = atom_b['en']
  en_c = atom_c['en']

  H_ab = get_deltaH_AinB(V_a, n_a, en_a, n_b, en_b, P, R)
  H_ba = get_deltaH_AinB(V_b, n_b, en_b, n_a, en_a, P, R)
  H_ac = get_deltaH_AinB(V_a, n_a, en_a, n_c, en_c, P, R)
  H_ca = get_deltaH_AinB(V_c, n_c, en_c, n_a, en_a, P, R)
  H_bc = get_deltaH_AinB(V_b, n_b, en_b, n_c, en_c, P, R)
  H_cb = get_deltaH_AinB(V_c, n_c, en_c, n_b, en_b, P, R)

  print('Solution Enthalpies: ', [H_ab, H_ba, H_ac, H_ca, H_bc, H_cb])

  x_ab = get_x_AinB(xa, xb)
  x_ba = get_x_AinB(xb, xa)
  x_ac = get_x_AinB(xa, xc)
  x_ca = get_x_AinB(xc, xa)
  x_bc = get_x_AinB(xb, xc)
  x_cb = get_x_AinB(xc, xb)

  CS_A = get_CS(xa, V_a, xb, V_b, xc, V_c)
  CS_B = get_CS(xb, V_b, xa, V_a, xc, V_c)
  CS_C = get_CS(xc, V_c, xa, V_a, xb, V_b)

  f = get_f(CS_A, CS_B, CS_C, delta) 

  H_chem = get_deltaH(f, x_ab, H_ab, x_ba, H_ba, x_ac, H_ac, x_ca, H_ca, x_bc, H_bc, x_cb, H_cb)

  # if delta == 0:
#
  #   H_ab_elast = get_deltaH_elast(atom_a, xa, atom_b, xb)
  #   H_ac_elast = get_deltaH_elast(atom_a, xa, atom_c, xb)
  #   H_bc_elast = get_deltaH_elast(atom_b, xb, atom_c, xc)

  #   H_struct_ab = get_H_struct(atom_a, xa, atom_b, xb)
  #   H_struct_ac = get_H_struct(atom_a, xa, atom_c, xc)
  #   H_struct_bc = get_H_struct(atom_b, xb, atom_c, xc)

  #   return H_chem + H_ab_elast + H_ac_elast + H_bc_elast + H_struct_ab + H_struct_ac + H_struct_bc
 # else:
  return H_chem 

## Store Atomic Parameters
See Table 1 for complete list

#### Additional Parameters 
`K`: Bulk Modulus (GPa) <br>
`G`: Shear Modulus (GPa) <br>
`N`: Num Valence Electrons <br>
`structure`: Crystal structure: `fcc`, `bcc`, `hcp`

In [3]:
#@title
Ag = {'Name': 'Ag', 'en': 4.45, 'n': 1.39, 'V': 4.7, 'R/P': 0.15,  'K': 84, 'G': 24, 'N': 11, 'structure': 'fcc', 'Transition_Metal': True}
Au = {'Name': 'Au', 'en': 5.15, 'n': 1.57, 'V': 4.7, 'R/P': 0.3, 'K': 148, 'G': 26, 'N': 11, 'structure': 'fcc', 'Transition_Metal': True}
Cu = {'Name': 'Cu', 'en': 4.55, 'n': 1.47, 'V': 3.7, 'R/P': 0.3, 'K': 130, 'G': 44, 'N': 11, 'structure': 'fcc', 'Transition_Metal': True}
Ce = {'Name': 'Ce', 'en': 3.18, 'n': 1.19, 'V': 7.8, 'R/P': 'NaN', 'K': 21.5, 'G': 13.5, 'N': 4, 'structure': 'fcc', 'Transition_Metal': False}
Fe = {'Name': 'Fe', 'en': 4.93, 'n': 1.77, 'V': 3.7, 'R/P': 1.0, 'K': 170, 'G': 84, 'N': 8, 'structure': 'fcc', 'Transition_Metal': True}
Ni = {'Name': 'Ni', 'en': 5.20, 'n': 1.75, 'V': 3.5, 'R/P': 1.0, 'K': 180, 'G': 86, 'N': 10, 'structure': 'fcc', 'Transition_Metal': True}
Nb = {'Name': 'Nb', 'en': 4.00, 'n': 1.62, 'V': 4.1, 'R/P': 1.0, 'K': 170, 'G': 38, 'N': 5, 'structure': 'bcc', 'Transition_Metal': True}
V = {'Name': 'V', 'en': 4.25, 'n': 1.64, 'V': 4.1, 'R/P': 1.0, 'K': 160, 'G': 44, 'N': 5, 'structure': 'bcc', 'Transition_Metal': True}
Zr = {'Name': 'Zr', 'en': 3.40, 'n': 1.39, 'V': 5.8, 'R/P': 1.0, 'K': 35.3, 'G': 53.4, 'N': 4, 'structure': 'hcp', 'Transition_Metal': True}
Al = {'Name': 'Al', 'en': 4.20, 'n': 1.39, 'V': 4.6, 'R/P': 1.9, 'K': 76, 'G': 34, 'N': 13, 'structure': 'fcc', 'Transition_Metal': False}
Hf = {'Name': 'Hf', 'en': 3.55, 'n': 1.43, 'V': 5.6, 'R/P': 1.0, 'K': 110, 'G': 30, 'N': 4, 'structure': 'hcp', 'Transition_Metal': True}
Ti = {'Name': 'Ti', 'en': 3.65, 'n': 1.47, 'V': 4.8, 'R/P': 1.0, 'K': 110, 'G': 45, 'N': 4, 'structure': 'hcp', 'Transition_Metal': True}
Pd = {'Name': 'Pd', 'en': 5.45, 'n': 1.67, 'V': 4.3, 'R/P': 1.0, 'K': 180, 'G': 44, 'N': 10, 'structure': 'fcc', 'Transition_Metal': True}
Pt = {'Name': 'Pt', 'en': 5.65, 'n': 1.78, 'V': 4.8, 'R/P': 1.0, 'K': 230, 'G': 61, 'N': 10, 'structure': 'fcc', 'Transition_Metal': True}

atomic_info = pd.DataFrame([Ag, Au, Cu, Ce, Fe, Ni, Nb, V, Zr, Al, Hf, Ti, Pd, Pt])
atomic_info

,Name,en,n,V,R/P,K,G,N,structure,Transition_Metal
0,Ag,4.45,1.39,4.7,0.15,84.0,24.0,11,fcc,True
1,Au,5.15,1.57,4.7,0.3,148.0,26.0,11,fcc,True
2,Cu,4.55,1.47,3.7,0.3,130.0,44.0,11,fcc,True
3,Ce,3.18,1.19,7.8,NaN,21.5,13.5,4,fcc,False
4,Fe,4.93,1.77,3.7,1.0,170.0,84.0,8,fcc,True
5,Ni,5.20,1.75,3.5,1.0,180.0,86.0,10,fcc,True
6,Nb,4.00,1.62,4.1,1.0,170.0,38.0,5,bcc,True
7,V,4.25,1.64,4.1,1.0,160.0,44.0,5,bcc,True
8,Zr,3.40,1.39,5.8,1.0,35.3,53.4,4,hcp,True
9,Al,4.20,1.39,4.6,1.9,76.0,34.0,13,fcc,False


## Store Constant Parameters in the Chemical Enthalpy Calculations
Table 2 in paper

In [4]:
#@title
Q_TT = 132.54
Q_TN = 115.62
Q_NN = 99.64
P_TT = 14.1
P_TN = 12.3
P_NN = 10.6


## Calculate $\Delta H$ for Ternary Alloy

Confirming the numbers from the paper:

| No. | Alloy System | $ΔH_{MMM}$ <br> Paper | $\Delta H_{MMM}$ <br> This Code| % Diff |
| --- | --- | --- | --- | --- |
|1| $Ag_{0.075} Au_{0.075} Cu_{0.850}$ | -2.50 | -1.966 | 21% |
|2| $Ag_{0.100} Au_{0.100} Cu_{0.800}$ | -3.20 | -2.514 | 21% |
|3| $Ag_{0.125} Au_{0.375} Cu_{0.500}$ | -5.53 | -4.512 | 18% |
|4| $Ag_{0.150} Au_{0.050} Cu_{0.800}$ | -3.32 | -2.556 | 23% |

In [5]:
result = get_H(Ag, 0.075, Au,0.075, Cu, 0.850, P_TT, 0, delta=8)
diff =(np.abs(result) - (2.5))/(0.01*2.5)
print(result)
print('Percent Err: ', diff)

Solution Enthalpies:  [-18.12062306367564, -18.12062306367564, 4.749748299692324, 3.739163555076936, -26.765293275000044, -21.07055002500004]
-1.966770733725218
Percent Err:  -21.329170650991276


# Older Approach (Without Tertiary Mixing)

In [6]:
def old_f(C_A, C_B, delta=8):
  result = C_A*C_B*(1+delta*((C_A*C_B)**2))
  return result

def C(xa, Va, xb, Vb):
  return (xa*Va)/(xa*Va + xb*Vb)

def old_deltH_AB(f, xa, Hab, xb, Hba):
  return f*(xa*Hab + xb*Hba)

def old_deltH(atomA, xa, atomB, xb, atomC, xc, P, R=0, delta=8):
  Va = atomA['V']
  Vb = atomB['V']
  Vc = atomC['V']

  na = atomA['n']
  nb = atomB['n']
  nc = atomC['n']

  ena = atomA['en']
  enb = atomB['en']
  enc = atomC['en']

  C_AB = C(xa, Va, xb, Vb)
  C_BA = C(xb, Vb, xa, Va)
  C_AC = C(xa, Va, xc, Vc)
  C_CA = C(xc, Vc, xa, Va)
  C_BC = C(xb, Vb, xc, Vc)
  C_CB = C(xc, Vc, xb, Vb)
  
  Hab = get_deltaH_AinB(Va, na, ena, nb, enb, P, R)
  Hba = get_deltaH_AinB(Vb, nb, enb, na, ena, P, R)
  Hac = get_deltaH_AinB(Va, na, ena, nc, enc, P, R)
  Hca = get_deltaH_AinB(Vc, nc, enc, na, ena, P, R)
  Hbc = get_deltaH_AinB(Vb, nb, enb, nc, enc, P, R)
  Hcb = get_deltaH_AinB(Vc, nc, enc, nb, enc, P, R)

  fab = old_f(C_AB, C_BA, delta)
  fac = old_f(C_AC, C_CA, delta)
  fbc = old_f(C_BC, C_CB, delta)

  E1 = fab*(xa*Hab + xb*Hba)
  E2 = fac*(xa*Hac + xc*Hca)
  E3 = fbc*(xb*Hbc + xc*Hcb)

  if delta == 0:

      H_ab_elast = get_deltaH_elast(atomA, xa, atomB, xb)
      H_ac_elast = get_deltaH_elast(atomA, xa, atomB, xb)
      H_bc_elast = get_deltaH_elast(atomA, xb, atomB, xc)
      H_struct_ab = get_H_struct(atomA, xa, atomB, xb)
      H_struct_ac = get_H_struct(atomA, xa, atomC, xc)
      H_struct_bc = get_H_struct(atomB, xb, atomC, xc)

  deltH_ab =  E1 +  E2 + E3 + H_ab_elast + H_ac_elast + H_bc_elast

  return deltH_ab

### Calculate ΔH for Ternary Alloy

In [7]:
old_deltH(Fe, 0.20, Ni, 0.23, V, 0.57, P_TT, 0, delta=0)

E1:  -0.6615249847291509
E2:  -4.095347036257703
E3:  -1.6794470533339863
H_struct:  1.1300000000000003
H_struct:  4.514999999999999
H_struct:  16.945


-6.381139189452208